In [1]:
import pandas as pd

In [42]:
rating=pd.read_csv('ratings.csv')
rating.drop(columns="timestamp", inplace=True)
rating['movieId'] = rating['movieId'].astype(int)
rating['userId'] = rating['userId'].astype(int)

In [49]:
movie=pd.read_csv('movies_metadata.csv')
movie = movie[['id','original_title','vote_count']]
movie = movie[movie['vote_count']>30]
movie.dropna(inplace=True)

KeyboardInterrupt: 

# Get unique user ids

In [44]:
userIds = rating[['userId']].copy()
userIds.drop_duplicates(inplace=True)
userIds = userIds.astype(str)
userIds.describe()

,userId
count,270896
unique,270896
top,147100
freq,1


# Get unique movie ids 

In [53]:
movieIds = movie[['id']].copy()
movieIds.drop_duplicates(inplace=True)
movieIds.to_csv('movieIds', index=False)

In [47]:
ur = rating.groupby(['userId'])['movieId','rating'].apply(lambda x: x.values.tolist())

In [48]:
ur.head

<bound method NDFrame.head of userId
1         [[110.0, 1.0], [147.0, 4.5], [858.0, 5.0], [12...
2         [[5.0, 3.0], [25.0, 3.0], [32.0, 2.0], [58.0, ...
3         [[480.0, 3.0], [500.0, 2.0], [527.0, 4.0], [85...
4         [[223.0, 4.0], [415.0, 4.0], [648.0, 4.0], [10...
5         [[7.0, 3.0], [157.0, 2.0], [296.0, 4.0], [541....
6         [[11.0, 3.0], [1198.0, 3.0], [2565.0, 4.0], [2...
7         [[293.0, 5.0], [318.0, 4.0], [2571.0, 4.0], [2...
8         [[1.0, 4.0], [7.0, 2.0], [44.0, 3.0], [47.0, 4...
9         [[1.0, 4.5], [150.0, 4.0], [342.0, 5.0], [497....
10        [[150.0, 4.0], [153.0, 3.0], [165.0, 4.0], [29...
11        [[32.0, 3.5], [34.0, 4.0], [47.0, 3.5], [110.0...
12        [[1.0, 4.0], [16.0, 4.0], [17.0, 4.0], [29.0, ...
13        [[1911.0, 2.0], [1965.0, 3.5], [2116.0, 4.5], ...
14        [[293.0, 2.0], [318.0, 2.5], [2571.0, 4.5], [2...
15        [[6.0, 4.0], [107.0, 3.5], [135.0, 0.5], [260....
16        [[32.0, 4.5], [50.0, 5.0], [111.0, 5.0], [198....
17 

In [50]:
ur.to_csv('userRatings')